<a href="https://colab.research.google.com/github/DineshGujjeti/Infosys_Springboard_CodeGenie/blob/main/Milestone4/code/milestone4_codeGenie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install Dependencies (Final)
print("📦 Installing all required packages...")

# Set up environment variables from Colab Secrets
import os
from google.colab import userdata

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['JWT_SECRET_KEY'] = userdata.get('JWT_SECRET_KEY')
os.environ['NGROK_TOKEN'] = userdata.get('NGROK_TOKEN')

# Install all required packages
!pip install --upgrade pip

# PyTorch
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Transformers, Accelerate, and BitsAndBytes for 4-bit loading
!pip install transformers accelerate bitsandbytes

# Streamlit and utilities (radon removed)
!pip install streamlit pyngrok PyJWT bcrypt pandas graphviz plotly-express psutil streamlit-autorefresh numpy

print("✅ All dependencies installed.")

📦 Installing all required packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 63.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 69.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 108.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.8/700.8 kB 39.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [streamlit-autorefresh]
✅ All dependencies installed.


In [ ]:
%%writefile app.py

import streamlit as st
import sqlite3
import bcrypt
import jwt
import os
import graphviz
import ast
import torch
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import time
import gc # Garbage Collector
import psutil # For system stats
import json # For copy-to-clipboard
import numpy as np # For network graph
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datetime import datetime, timedelta
from streamlit_autorefresh import st_autorefresh # For live updates

# --- 1. CONFIGURATION & SECRETS ---

HF_TOKEN = os.environ.get('HF_TOKEN')
JWT_SECRET = os.environ.get('JWT_SECRET_KEY', "dummy_jwt")
NGROK_TOKEN = os.environ.get('NGROK_TOKEN', "dummy_ngrok")

DB_NAME = "users.db"

# --- 2. DATABASE UTILITIES ---

def init_db():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL,
        security_question TEXT NOT NULL,
        security_answer_hash TEXT NOT NULL,
        role TEXT NOT NULL DEFAULT 'user'
    )
    ''')
    c.execute('''
    CREATE TABLE IF NOT EXISTS activity_log (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        timestamp TEXT NOT NULL,
        prompt TEXT NOT NULL,
        model_name TEXT NOT NULL,
        language TEXT NOT NULL,
        response_time REAL,
        feedback_score INTEGER DEFAULT 0,
        feedback_comment TEXT
    )
    ''')
    c.execute("SELECT COUNT(*) FROM users")
    user_count = c.fetchone()[0]
    conn.commit()
    conn.close()
    return user_count == 0

def hash_text(text):
    salt = bcrypt.gensalt()
    return bcrypt.hashpw(text.encode('utf-8'), salt)

def check_hash(text, hashed):
    return bcrypt.checkpw(text.encode('utf-8'), hashed)

def add_user(username, password, question, answer, role='user'):
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        password_hash = hash_text(password)
        answer_hash = hash_text(answer)
        c.execute(
            "INSERT INTO users (username, password_hash, security_question, security_answer_hash, role) VALUES (?, ?, ?, ?, ?)",
            (username, password_hash, question, answer_hash, role)
        )
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()

def verify_user(username, password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT password_hash, role FROM users WHERE username=?", (username,))
    user = c.fetchone()
    conn.close()
    if user:
        password_hash, role = user
        if check_hash(password, password_hash):
            return {"username": username, "role": role}
    return None

def update_user_password(username, new_password):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    new_password_hash = hash_text(new_password)
    c.execute("UPDATE users SET password_hash = ? WHERE username = ?", (new_password_hash, username))
    conn.commit()
    conn.close()

def get_all_users():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT id, username, role FROM users")
    users = c.fetchall()
    conn.close()
    return users

def get_user_details(username):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT username, security_question, security_answer_hash FROM users WHERE username=?", (username,))
    user = c.fetchone()
    conn.close()
    if user:
        return {"username": user[0], "question": user[1], "answer_hash": user[2]}
    return None

# --- LOGGING AND QUERY FUNCTIONS ---

# --- THIS IS THE FIX ---
def log_user_activity(username, prompt, model_name, language, response_time):
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute("SELECT id FROM users WHERE username=?", (username,))
        user_id = c.fetchone()
        if user_id:
            user_id = user_id[0]
        else:
            return
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        c.execute(
            "INSERT INTO activity_log (user_id, timestamp, prompt, model_name, language, response_time) VALUES (?, ?, ?, ?, ?, ?)",
            (user_id, timestamp, prompt, model_name, language, response_time)
        )
        # --- END FIX ---
        conn.commit()
        c.execute("SELECT last_insert_rowid()")
        st.session_state['last_activity_id'] = c.fetchone()[0]
    except Exception as e:
        print(f"Error logging activity to DB: {e}")
    finally:
        if conn:
            conn.close()

def update_feedback_score(activity_id, score, comment):
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute("UPDATE activity_log SET feedback_score = ?, feedback_comment = ? WHERE id = ?", (score, comment, activity_id))
        conn.commit()

        add_system_log(f"Feedback (ID: {activity_id}): {score}⭐ - '{comment}'", "💬")

        if 'last_activity_id' in st.session_state:
            del st.session_state['last_activity_id']
        st.toast(f"Feedback submitted! Thank you.")
    except Exception as e:
        st.error(f"Failed to submit feedback: {e}")
    finally:
        if conn:
            conn.close()

# --- DASHBOARD QUERY FUNCTIONS ---
def get_total_users():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT COUNT(id) FROM users")
    count = c.fetchone()[0]
    conn.close()
    return count

def get_total_queries():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT COUNT(id) FROM activity_log")
    count = c.fetchone()[0]
    conn.close()
    return count

def get_average_latency():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT AVG(response_time) FROM activity_log")
    avg_latency = c.fetchone()[0]
    conn.close()
    return f"{avg_latency:.1f}" if avg_latency is not None else "0.0"

def get_trending_queries():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("""
        SELECT prompt, COUNT(prompt) as count
        FROM activity_log
        GROUP BY prompt
        ORDER BY count DESC
        LIMIT 5
    """)
    data = c.fetchall()
    conn.close()
    trending_data = [
        {'Query': (row[0][:40] + '...') if len(row[0]) > 40 else row[0], 'Count': row[1]}
        for row in data
    ]
    trending_data = pd.DataFrame(trending_data)
    return trending_data

def get_language_usage():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("""
        SELECT language, COUNT(language) as count
        FROM activity_log
        GROUP BY language
    """)
    data = c.fetchall()
    conn.close()
    df = pd.DataFrame(data, columns=['Language', 'Count'])
    if df.empty:
        return pd.DataFrame({'Language': ['None'], 'Count': [0]})
    df = df.sort_values(by='Count', ascending=False)
    return df

def get_positive_feedback_rate():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT COUNT(id) FROM activity_log WHERE feedback_score != 0")
    total_feedback = c.fetchone()[0]
    if total_feedback == 0:
        conn.close()
        return "N/A", None
    c.execute("SELECT COUNT(id) FROM activity_log WHERE feedback_score > 3") # 4 or 5 stars
    positive_feedback = c.fetchone()[0]
    rate = (positive_feedback / total_feedback) * 100
    conn.close()
    return f"{rate:.0f}%", None

def get_user_history(username):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT id FROM users WHERE username=?", (username,))
    user_id = c.fetchone()[0]
    c.execute("""
        SELECT timestamp, model_name, language, prompt, response_time, feedback_score
        FROM activity_log
        WHERE user_id = ?
        ORDER BY timestamp DESC
        LIMIT 10
    """, (user_id,))
    data = c.fetchall()
    conn.close()
    history = pd.DataFrame(data, columns=['Timestamp', 'Model', 'Lang', 'Prompt', 'Latency (s)', 'Feedback'])
    history['Latency (s)'] = history['Latency (s)'].round(2)
    history['Feedback'] = history['Feedback'].map(lambda x: "⭐" * x if x > 0 else "—")
    return history

def get_full_history():
    conn = sqlite3.connect(DB_NAME)
    history_df = pd.read_sql_query("SELECT * FROM activity_log ORDER BY timestamp DESC LIMIT 20", conn)
    conn.close()
    return history_df

def get_full_feedback():
    conn = sqlite3.connect(DB_NAME)
    query = """
    SELECT
        a.timestamp,
        u.username,
        a.prompt,
        a.model_name,
        a.feedback_score,
        a.feedback_comment
    FROM activity_log a
    JOIN users u ON a.user_id = u.id
    WHERE a.feedback_score > 0
    ORDER BY a.timestamp DESC
    LIMIT 10
    """
    feedback_df = pd.read_sql_query(query, conn)
    conn.close()

    feedback_df['Feedback'] = feedback_df['feedback_score'].map(lambda x: "⭐" * x)
    return feedback_df[['timestamp', 'username', 'prompt', 'Feedback', 'feedback_comment']]


# --- 3. AUTHENTICATION (JWT) ---

def create_jwt(username, role):
    payload = {
        'sub': username,
        'role': role,
        'iat': datetime.utcnow(),
        'exp': datetime.utcnow() + timedelta(hours=24)
    }
    token = jwt.encode(payload, JWT_SECRET, algorithm='HS256')
    return token

def verify_jwt(token):
    try:
        payload = jwt.decode(token, JWT_SECRET, algorithms=['HS256'])
        return payload
    except jwt.ExpiredSignatureError:
        st.error("Session expired. Please log in again.")
        return None
    except jwt.InvalidTokenError:
        st.error("Invalid token. Please log in again.")
        return None

# --- 4. DYNAMIC MODEL LOADING (MEMORY EFFICIENT) ---

MODELS = {
    "Gemma-2B-IT": "google/gemma-2b-it",
    "DeepSeek-Coder-1.3B": "deepseek-ai/deepseek-coder-1.3b-instruct",
    "Phi-2": "microsoft/phi-2"
}

def load_model_from_hub(model_path):
    token = os.environ.get('HF_TOKEN')
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, token=token)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        quantization_config=quantization_config,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        token=token
    )
    return model, tokenizer

def get_model_and_tokenizer(model_name_key):
    model_path = MODELS.get(model_name_key)
    if not model_path:
        st.error(f"Model key {model_name_key} not found in MODELS dict.")
        return None, None

    if st.session_state.get('loaded_model_name') != model_path:
        if 'loaded_model' in st.session_state:
            log_msg = f"Unloading old model: {st.session_state.loaded_model_name}"
            print(log_msg)
            add_system_log(log_msg, "♻️")
            del st.session_state.loaded_model
            del st.session_state.loaded_tokenizer
            gc.collect()
            torch.cuda.empty_cache()
            print("Old model unloaded.")

        log_msg = f"Loading new model: {model_path}..."
        print(log_msg)
        add_system_log(log_msg, "⏳")

        with st.spinner(f"Loading {model_name_key}... (This may take a moment)"):
            try:
                start_load = time.time()
                model, tokenizer = load_model_from_hub(model_path)
                load_time = time.time() - start_load
            except Exception as e:
                st.error(f"Failed to load model {model_path}: {e}")
                if 'loaded_model_name' in st.session_state: del st.session_state.loaded_model_name
                return None, None

        st.session_state.loaded_model_name = model_path
        st.session_state.loaded_model = model
        st.session_state.loaded_tokenizer = tokenizer
        st.session_state.model_load_time = load_time
        log_msg = f"Model {model_name_key} loaded in {load_time:.2f}s."
        print(log_msg)
        add_system_log(log_msg, "✅")

    else:
        print(f"Using cached model: {model_path}")

    return st.session_state.loaded_model, st.session_state.loaded_tokenizer

def generate_code(prompt, model_name_key):
    model, tokenizer = get_model_and_tokenizer(model_name_key)
    if model is None:
        return f"Error: Model {model_name_key} failed to load."

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    attention_mask = inputs.attention_mask
    output_ids = model.generate(
        inputs.input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    response_tokens = output_ids[0][inputs.input_ids.shape[-1]:]
    code = tokenizer.decode(response_tokens, skip_special_tokens=True)

    if not code or not code.strip() or "undefined" in code:
        return "```\nError: Model returned an empty or invalid response. Please try again.\n```"

    formatted_code = f"```python\n{code.strip()}\n```"
    return formatted_code

def query_hf_model(payload, model_name_key):
    try:
        formatted_code = generate_code(payload, model_name_key)
        return formatted_code
    except Exception as e:
        return f"Error: Code Generation/Explanation failed: {str(e)}"

# --- 5. AST VISUALIZER ---

class ASTVisualizer:
    def __init__(self):
        self.graph = graphviz.Digraph(comment='Python AST', graph_attr={'rankdir': 'TB', 'bgcolor': '#333333'}, node_attr={'style': 'filled', 'color': '#FF6347', 'fontcolor': 'white', 'fillcolor': '#444444'})

    def traverse(self, node, parent_id=None):
        node_id = str(id(node))
        node_label = type(node).__name__
        if hasattr(node, 'id') and isinstance(node.id, str):
            node_label = f"Name\n(id='{node.id}')"
        elif isinstance(node, ast.Constant):
            node_label = f"Constant\n(value={repr(node.value)})"
        elif isinstance(node, ast.Attribute):
            node_label = f"Attribute\n(attr='{node.attr}')"
        elif isinstance(node, ast.FunctionDef):
            node_label = f"FunctionDef\n(name='{node.name}')"
        self.graph.node(node_id, label=node_label)
        if parent_id:
            self.graph.edge(parent_id, node_id)
        for field, value in ast.iter_fields(node):
            if isinstance(value, list):
                for item in value:
                    if isinstance(item, ast.AST):
                        self.traverse(item, node_id)
            elif isinstance(value, ast.AST):
                self.traverse(value, node_id)

    def get_graph(self, code):
        try:
            tree = ast.parse(code)
            self.traverse(tree)
            return self.graph
        except SyntaxError as e:
            st.error(f"Python Syntax Error: {e}")
            return None
        except Exception as e:
            st.error(f"Error building AST: {e}")
            return None

# --- 6. STREAMLIT APP UI ---

def add_system_log(message, icon="⚙️"):
    """Adds a new log to the session state log list."""
    if 'system_logs' not in st.session_state:
        st.session_state.system_logs = []
    log_entry = f"{icon} {datetime.now().strftime('%H:%M:%S')} - {message}"
    st.session_state.system_logs.insert(0, log_entry)
    st.session_state.system_logs = st.session_state.system_logs[:50]

def initialize_state():
    # Base state
    if "logged_in" not in st.session_state: st.session_state.logged_in = False
    if "username" not in st.session_state: st.session_state.username = None
    if "role" not in st.session_state: st.session_state.role = None
    if "page" not in st.session_state: st.session_state.page = "Login"
    if "token" not in st.session_state: st.session_state.token = None
    if "chat_history" not in st.session_state: st.session_state.chat_history = []
    if "last_output" not in st.session_state: st.session_state.last_output = ""
    if "last_activity_id" not in st.session_state: st.session_state.last_activity_id = None
    if "reset_step" not in st.session_state: st.session_state.reset_step = 1
    if "reset_user" not in st.session_state: st.session_state.reset_user = None

    # DYNAMIC MODEL LOADING STATE
    if 'loaded_model_name' not in st.session_state: st.session_state.loaded_model_name = None
    if 'loaded_model' not in st.session_state: st.session_state.loaded_model = None
    if 'loaded_tokenizer' not in st.session_state: st.session_state.loaded_tokenizer = None
    if 'model_load_time' not in st.session_state: st.session_state.model_load_time = 0.0

    # Explainer state
    if 'explanation' not in st.session_state: st.session_state.explanation = ""
    if 'ast_graph' not in st.session_state: st.session_state.ast_graph = None

    # --- NEW ADMIN DASHBOARD STATE ---
    if 'admin_boot_animation_done' not in st.session_state: st.session_state.admin_boot_animation_done = False
    if 'gpu_history' not in st.session_state: st.session_state.gpu_history = [] # For live charts
    if 'network_history' not in st.session_state: st.session_state.network_history = [] # For live charts
    if 'system_logs' not in st.session_state: st.session_state.system_logs = []
    if 'exp_torch_compile' not in st.session_state: st.session_state.exp_torch_compile = False
    if 'exp_mixed_precision' not in st.session_state: st.session_state.exp_mixed_precision = True
    if 'reveal_keys' not in st.session_state: st.session_state.reveal_keys = False


def show_login_page(is_first_user):
    st.title("CodeGenie 🧞")
    col1, col2 = st.columns([2, 3])
    with col1:
        st.subheader("Welcome Back")
    with col2:
        with st.container(border=True):
            tab1, tab2 = st.tabs(["Login", "Sign Up"])
            with tab1:
                with st.form("login_form"):
                    username = st.text_input("Username", key="login_username")
                    password = st.text_input("Password", type="password", key="login_password")
                    login_button = st.form_submit_button("Login", use_container_width=True)
                    if login_button:
                        user_data = verify_user(username, password)
                        if user_data:
                            st.session_state.logged_in = True
                            st.session_state.username = user_data["username"]
                            st.session_state.role = user_data["role"]
                            st.session_state.token = create_jwt(user_data["username"], user_data["role"])
                            if user_data["role"] == 'admin':
                                st.session_state.page = "Dashboard"
                            else:
                                st.session_state.page = "Generator"

                            add_system_log(f"User '{user_data['username']}' logged in.", "👤")
                            st.rerun()
                        else:
                            st.error("Invalid username or password")
                if st.button("Forgot Password?", type="primary"):
                    st.session_state.page = "Forgot Password"
                    st.rerun()
            with tab2:
                with st.form("signup_form"):
                    st.markdown("### Create Your Account")
                    if is_first_user:
                        st.info("✨ You will be the first user! You will be registered as an **Admin**.")
                    new_username = st.text_input("Username*")
                    new_password = st.text_input("Password*", type="password")
                    new_password_confirm = st.text_input("Confirm Password*", type="password")
                    st.markdown("---")
                    st.markdown("**Security Question (for password reset)**")
                    question = st.selectbox("Select a question*", ["What was the name of your first pet?", "What is your mother's maiden name?", "What city were you born in?", "What was your first car?"])
                    answer = st.text_input("Your Answer*", type="password")

                    signup_button = st.form_submit_button("Sign Up", use_container_width=True)
                    if signup_button:
                        if not (new_username and new_password and new_password_confirm and answer):
                            st.error("Please fill out all required fields.")
                        elif new_password != new_password_confirm:
                            st.error("Passwords do not match.")
                        else:
                            role = 'admin' if is_first_user else 'user'
                            success = add_user(new_username, new_password, question, answer, role)
                            if success:
                                add_system_log(f"New user '{new_username}' registered as '{role}'.", "➕")
                                st.success(f"User '{new_username}' created successfully as {'an Admin' if role == 'admin' else 'a User'}. Please log in.")
                            else:
                                st.error("Username already exists.")

def show_forgot_password_page():
    st.title("Reset Password")
    with st.container(border=True):
        if "reset_user" not in st.session_state: st.session_state.reset_user = None
        if "reset_step" not in st.session_state: st.session_state.reset_step = 1

        if st.session_state.reset_step == 1:
            with st.form("step1_username"):
                username = st.text_input("Enter your username")
                submit_user = st.form_submit_button("Next")
                if submit_user:
                    user_details = get_user_details(username)
                    if user_details:
                        st.session_state.reset_user = user_details
                        st.session_state.reset_step = 2
                        st.rerun()
                    else:
                        st.error("Username not found.")
        if st.session_state.reset_step == 2:
            st.info(f"**Security Question:** {st.session_state.reset_user['question']}")
            with st.form("step2_answer"):
                answer = st.text_input("Your Answer", type="password")
                submit_answer = st.form_submit_button("Verify Answer")
                if submit_answer:
                    if check_hash(answer, st.session_state.reset_user['answer_hash']):
                        st.session_state.reset_step = 3
                        st.rerun()
                    else:
                        st.error("Incorrect answer.")
        if st.session_state.reset_step == 3:
            st.success("Verification successful!")
            with st.form("step3_new_password"):
                new_password = st.text_input("New Password", type="password")
                confirm_new_password = st.text_input("Confirm New Password", type="password")
                submit_pass = st.form_submit_button("Reset Password")
                if submit_pass:
                    if not new_password or new_password != confirm_new_password:
                        st.error("Passwords do not match or are empty.")
                    else:
                        update_user_password(st.session_state.reset_user['username'], new_password)
                        add_system_log(f"User '{st.session_state.reset_user['username']}' reset password.", "🔑")
                        st.success("Password reset successfully! You can now log in.")
                        st.session_state.reset_step = 1
                        st.session_state.reset_user = None
                        st.session_state.page = "Login"
                        st.rerun()
    if st.button("← Back to Login"):
        st.session_state.page = "Login"
        st.session_state.reset_step = 1
        st.session_state.reset_user = None
        st.rerun()

def preload_selected_model():
    if 'gen_model_key' in st.session_state and st.session_state.gen_model_key:
        print(f"Pre-loading model: {st.session_state.gen_model_key}")
        get_model_and_tokenizer(st.session_state.gen_model_key)

def show_generator_page():
    st.title("🤖 Code Generator")
    model_keys = list(MODELS.keys())
    selected_model = st.selectbox(
        "Select AI Model",
        model_keys,
        key='gen_model_key',
        on_change=preload_selected_model
    )

    if 'loaded_model' not in st.session_state or st.session_state.loaded_model is None:
        preload_selected_model()

    chat_container = st.container(height=400, border=True)
    with chat_container:
        for entry in st.session_state.chat_history:
            with st.chat_message(entry["role"]):
                st.markdown(entry["content"])
    prompt = st.text_area("Your prompt:", key="prompt_input", height=100)
    col1, col2, col3 = st.columns([2, 1, 1])

    with col1:
        if st.button("Generate Code 🚀", use_container_width=True, type="primary"):
            if prompt:
                st.session_state.chat_history.append({"role": "user", "content": prompt})
                with chat_container:
                    with st.chat_message("user"):
                        st.markdown(prompt)
                    with st.chat_message("assistant"):
                         message_placeholder = st.empty()

                with st.spinner(f"CodeGenie is thinking with {selected_model}..."):
                    start_time = time.time()
                    try:
                        response, complexity_score = generate_code(prompt, selected_model)
                    except Exception as e:
                        response, complexity_score = f"Error: {str(e)}", 0
                    end_time = time.time()
                    latency = end_time - start_time

                log_user_activity(st.session_state.username, prompt, selected_model, "Python", latency, complexity_score)
                message_placeholder.markdown(response)
                st.session_state.chat_history.append({"role": "assistant", "content": response})
                if not response.startswith("Error:"):
                    st.session_state.last_output = response
                st.rerun()
            else:
                st.warning("Please enter a prompt.")

    with col2:
        if st.button("📋 Copy Last Output", use_container_width=True):
            if st.session_state.last_output:
                st.components.v1.html(f"<script>navigator.clipboard.writeText({repr(st.session_state.last_output)});</script>", height=0)
                st.toast("Output copied to clipboard!")
            else:
                st.toast("No output to copy yet.")
    with col3:
        if st.button("🗑️ Clear History", use_container_width=True):
            st.session_state.chat_history = []
            st.session_state.last_output = ""
            st.session_state.explanation = ""
            st.session_state.ast_graph = None
            st.rerun()

    if 'last_activity_id' in st.session_state and st.session_state.last_activity_id is not None:
        st.markdown("---")
        with st.form(key="feedback_form_gen"):
            st.subheader("Was this response helpful?")
            rating = st.radio(
                "Rate this response:",
                options=[1, 2, 3, 4, 5],
                format_func=lambda x: "⭐" * x,
                horizontal=True
            )
            comment = st.text_area("Additional feedback (optional):")

            if st.form_submit_button("Submit Feedback"):
                try:
                    update_feedback_score(st.session_state.last_activity_id, rating, comment)
                    st.success("Thank you for your feedback!")
                    st.session_state.last_activity_id = None
                    st.rerun()
                except Exception as e:
                    st.error(f"Failed to submit feedback: {e}")

def preload_explainer_model():
    if 'exp_model_key' in st.session_state and st.session_state.exp_model_key:
        print(f"Pre-loading explainer model: {st.session_state.exp_model_key}")
        get_model_and_tokenizer(st.session_state.exp_model_key)

def show_explainer_page():
    st.title("🔬 Code Explainer & AST Visualizer")
    lang = st.selectbox("Select Language", ["Python", "JavaScript", "SQL"])
    code = st.text_area("Paste your code here:", height=250)
    model_keys = list(MODELS.keys())
    explainer_model_name = st.selectbox(
        "Select Explainer Model",
        model_keys,
        index=0,
        key='exp_model_key',
        on_change=preload_explainer_model
    )

    if 'loaded_model' not in st.session_state or st.session_state.loaded_model is None:
        preload_explainer_model()

    if st.button("Explain & Visualize", type="primary"):
        if not code:
            st.warning("Please paste some code to explain.")
            st.session_state.explanation = ""
            st.session_state.ast_graph = None
            return

        st.session_state.explanation = ""
        st.session_state.ast_graph = None

        with st.spinner(f"Generating explanation with {explainer_model_name}..."):
            explain_prompt = f"Explain the following {lang} code. Describe what it does, step by step. Provide the explanation in markdown format, using code blocks for the code.\n\n```\n{code}\n```"
            start_time = time.time()
            try:
                explanation, complexity_score = query_hf_model(explain_prompt, explainer_model_name)
            except Exception as e:
                explanation, complexity_score = f"Error: {str(e)}", 0
            end_time = time.time()
            latency = end_time - start_time
            log_user_activity(st.session_state.username, code, explainer_model_name, lang, latency, complexity_score)
            st.session_state.explanation = explanation

        if lang == "Python":
            visualizer = ASTVisualizer()
            graph = visualizer.get_graph(code)
            if graph:
                st.session_state.ast_graph = graph

        st.session_state.last_output = "Explainer Run"

    if st.session_state.explanation:
        col1, col2 = st.columns(2)
        with col1:
            st.subheader("AI Explanation")
            st.markdown(st.session_state.explanation)
        with col2:
            st.subheader("Abstract Syntax Tree (AST)")
            if st.session_state.ast_graph:
                st.graphviz_chart(st.session_state.ast_graph)
            elif lang == "Python":
                 st.error("Could not generate AST for this Python code.")
            else:
                st.info(f"AST visualization is only available for Python.")

    if 'last_activity_id' in st.session_state and st.session_state.last_activity_id is not None and st.session_state.last_output == "Explainer Run":
        st.markdown("---")
        with st.form(key="feedback_form_exp"):
            st.subheader("Was this explanation helpful?")
            rating = st.radio(
                "Rate this response:",
                options=[1, 2, 3, 4, 5],
                format_func=lambda x: "⭐" * x,
                horizontal=True
            )
            comment = st.text_area("Additional feedback (optional):")

            if st.form_submit_button("Submit Feedback"):
                try:
                    update_feedback_score(st.session_state.last_activity_id, rating, comment)
                    st.success("Thank you for your feedback!")
                    st.session_state.last_activity_id = None
                    st.rerun()
                except Exception as e:
                    st.error(f"Failed to submit feedback: {e}")


# --- 9. USER PROFILE PAGE ---

def show_profile_page():
    st.title(f"Profile: {st.session_state.username}")
    st.info(f"**Username:** `{st.session_state.username}`\n\n**Role:** `{st.session_state.role}`")
    st.subheader("Change Password")
    with st.form("change_password_form"):
        current_password = st.text_input("Current Password", type="password")
        new_password = st.text_input("New Password", type="password")
        confirm_new_password = st.text_input("Confirm New Password", type="password")
        submit_change = st.form_submit_button("Update Password")
        if submit_change:
            user_data = verify_user(st.session_state.username, current_password)
            if not user_data:
                st.error("Incorrect current password.")
            elif not new_password or new_password != confirm_new_password:
                st.error("New passwords do not match or are empty.")
            else:
                update_user_password(st.session_state.username, new_password)
                add_system_log(f"User '{st.session_state.username}' changed password.", "🔑")
                st.success("Password updated successfully!")

    st.markdown("---")
    st.subheader("Recent Activity History")
    history_df = get_user_history(st.session_state.username)
    if not history_df.empty:
        st.dataframe(history_df, use_container_width=True, hide_index=True)
    else:
        st.info("No activity logged yet.")

# --- 10. FUTURISTIC ADMIN DASHBOARD ---

def get_gpu_stats():
    """Gets GPU stats from torch.cuda."""
    if not torch.cuda.is_available():
        return "N/A (CPU)", 0, 0, "N/A", "N/A"

    gpu_name = torch.cuda.get_device_name(0)
    total_mem = torch.cuda.get_device_properties(0).total_memory
    reserved_mem = torch.cuda.memory_reserved(0)
    alloc_mem = torch.cuda.memory_allocated(0)
    free_mem = total_mem - reserved_mem

    used_mem = total_mem - free_mem
    util = (used_mem / total_mem) * 100

    return gpu_name, total_mem / 1e9, used_mem / 1e9, f"{util:.1f}%", torch.version.cuda

def get_system_uptime():
    """Gets system uptime using psutil."""
    try:
        boot_time = psutil.boot_time()
        uptime_seconds = time.time() - boot_time
        uptime = str(timedelta(seconds=int(uptime_seconds)))
        return uptime
    except Exception:
        return "N/A"

def show_admin_dashboard_page():
    if st.session_state.role != 'admin':
        st.error("You do not have permission to view this page.")
        return

    # --- Cyberpunk CSS ---
    st.markdown("""
        <style>
            .stApp { background-color: #0F111A; }
            h1 {
                color: #00F0FF;
                text-shadow: 0 0 10px #00F0FF, 0 0 20px #00F0FF;
                font-family: 'Courier New', Courier, monospace;
            }
            h2, h3 {
                color: #FF00FF;
                text-shadow: 0 0 5px #FF00FF;
                font-family: 'Courier New', Courier, monospace;
                border-bottom: 1px solid #FF00FF;
                padding-bottom: 5px;
            }
            .glowing-card {
                padding: 18px; border-radius: 10px; border: 1px solid #00F0FF;
                box-shadow: 0 0 15px #00F0FF; background-color: #1A1C2A; margin-bottom: 10px;
            }
            .glowing-card-magenta { border: 1px solid #FF00FF; box-shadow: 0 0 15px #FF00FF; }
            .glowing-card div[data-testid="stMetricLabel"] {
                color: #00F0FF; font-size: 1.1em; font-family: 'Courier New', Courier, monospace;
            }
            .glowing-card div[data-testid="stMetricValue"] {
                color: #FFFFFF; font-size: 2.5em; font-family: 'Courier New', Courier, monospace;
            }
            .log-entry {
                font-family: 'IBM Plex Mono', monospace; font-size: 0.9em; color: #E0E0E0;
                border-bottom: 1px solid #333; padding-bottom: 5px; margin-bottom: 5px;
            }
            .health-badge {
                display: inline-block; padding: 4px 10px; border-radius: 15px;
                font-size: 0.9em; font-family: 'Courier New', Courier, monospace; margin-right: 10px;
            }
            .health-green { background-color: #004D00; color: #39FF14; border: 1px solid #39FF14; }
            .health-red { background-color: #4D0000; color: #FF073A; border: 1px solid #FF073A; }
            .health-yellow { background-color: #4D4D00; color: #FFFF33; border: 1px solid #FFFF33; }
        </style>
    """, unsafe_allow_html=True)

    # --- 1. Cyberpunk Boot Animation ---
    if not st.session_state.admin_boot_animation_done:
        st.title("CodeGenie Control Grid")
        boot_placeholder = st.empty()
        boot_messages = [
            "⚙️ Initializing Neural Interface...",
            "🔌 Syncing model cores...",
            "🌐 Activating CodeGenie Control Grid...",
            "✅ System Online. Welcome, Admin."
        ]
        message_so_far = ""
        for msg in boot_messages:
            message_so_far += f"<h3 style='font-family: Courier New; color: #39FF14; text-shadow: 0 0 10px #39FF14;'>{msg}</h3>"
            boot_placeholder.markdown(message_so_far, unsafe_allow_html=True)
            time.sleep(0.5)
        time.sleep(1)
        st.session_state.admin_boot_animation_done = True
        add_system_log("Admin Dashboard Initialized", "🚀")
        st.rerun()

    # --- Main Dashboard ---
    st.title("👑 Admin Dashboard")
    st_autorefresh(interval=10000, key="admin_refresh")

    gpu_name, total_mem, used_mem, util, cuda_v = get_gpu_stats()

    if total_mem > 0 and (used_mem / total_mem) > 0.9:
        st.error(f"🚨 VRAM usage is critical: {util}! System may be unstable.", icon="🚨")

    # --- NEW: Replaced Model Monitor with Key Stats ---
    dash_col1, dash_col2 = st.columns(2)
    with dash_col1:
        st.subheader("📊 Key Usage Statistics")
        total_users = get_total_users()
        total_queries = get_total_queries()
        avg_latency = get_average_latency()
        feedback_rate, feedback_delta = get_positive_feedback_rate()

        c1, c2 = st.columns(2)
        with c1:
            st.markdown(f"""
            <div class="glowing-card">
                <div data-testid="stMetricLabel">Total Users</div>
                <div data-testid="stMetricValue" style="font-size: 2em;">{total_users}</div>
            </div>
            """, unsafe_allow_html=True)
            st.markdown(f"""
            <div class="glowing-card glowing-card-magenta">
                <div data-testid="stMetricLabel">Avg. Response Time</div>
                <div data-testid="stMetricValue" style="font-size: 2em; color: #FF00FF;">{avg_latency}s</div>
            </div>
            """, unsafe_allow_html=True)
        with c2:
            st.markdown(f"""
            <div class="glowing-card">
                <div data-testid="stMetricLabel">Total Queries</div>
                <div data-testid="stMetricValue" style="font-size: 2em;">{total_queries}</div>
            </div>
            """, unsafe_allow_html=True)
            st.markdown(f"""
            <div class="glowing-card glowing-card-magenta">
                <div data-testid="stMetricLabel">Positive Feedback</div>
                <div data-testid="stMetricValue" style="font-size: 2em; color: #FF00FF;">{feedback_rate}</div>
            </div>
            """, unsafe_allow_html=True)

    with dash_col2:
        st.subheader("📡 Live Network & Query Ticker")

        if 'network_history' not in st.session_state: st.session_state.network_history = []
        new_traffic = np.random.randint(50, 500)
        st.session_state.network_history.append({"time": datetime.now(), "Traffic (MB/s)": new_traffic})
        st.session_state.network_history = st.session_state.network_history[-50:]

        df_net = pd.DataFrame(st.session_state.network_history)
        fig_net = go.Figure()
        fig_net.add_trace(go.Scatter(x=df_net['time'], y=df_net['Traffic (MB/s)'], mode='lines',
                                     line=dict(color='#39FF14', width=2), fill='tozeroy',
                                     fillcolor='rgba(57, 255, 20, 0.3)'))
        fig_net.update_layout(
            title="Simulated Network I/O", template="plotly_dark", paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(26, 28, 42, 0.8)', font_color="#39FF14", height=200,
            xaxis=dict(showgrid=False, zeroline=False), yaxis=dict(gridcolor='#444', zeroline=False)
        )
        st.plotly_chart(fig_net, use_container_width=True, config={'displayModeBar': False})

        st.markdown("<h6>Live Query Ticker</h6>", unsafe_allow_html=True)
        ticker_logs = get_full_history().head(5) # Get last 5 queries
        ticker_placeholder = st.empty()
        log_html = ""
        for _, row in ticker_logs.iterrows():
            log_html += f"<div class='log-entry'>🛰️ {row['timestamp']} - User ID {row['user_id']} queried {row['model_name']}</div>"
        ticker_placeholder.markdown(f"""
            <div style="height: 150px; background-color: #0A0C12; border-radius: 5px; padding: 10px; overflow-y: scroll; border: 1px solid #444;">
                {log_html if not ticker_logs.empty else "<div class='log-entry'>...Awaiting user activity...</div>"}
            </div>
        """, unsafe_allow_html=True)

    st.markdown("---")

    # --- System Health & VRAM Graph (Moved to bottom) ---
    col1, col2 = st.columns([1, 2])
    with col1:
        st.subheader("System Health")
        st.markdown("<h5>Health Check</h5>", unsafe_allow_html=True)
        if torch.cuda.is_available():
            st.markdown('<span class="health-badge health-green">🟢 CUDA: OK</span>', unsafe_allow_html=True)
        else:
            st.markdown('<span class="health-badge health-red">🔴 CUDA: Fail</span>', unsafe_allow_html=True)
        if HF_TOKEN and HF_TOKEN.startswith("hf_"):
            st.markdown('<span class="health-badge health-green">🟢 HF Token: OK</span>', unsafe_allow_html=True)
        else:
            st.markdown('<span class="health-badge health-red">🔴 HF Token: Missing</span>', unsafe_allow_html=True)
        if NGROK_TOKEN:
            st.markdown('<span class="health-badge health-green">🟢 Ngrok Token: OK</span>', unsafe_allow_html=True)
        else:
            st.markdown('<span class="health-badge health-red">🔴 Ngrok Token: Missing</span>', unsafe_allow_html=True)

        st.markdown("---")
        st.markdown(f"""
            <div class="glowing-card">
                <div data-testid="stMetricLabel">GPU</div>
                <div data-testid="stMetricValue" style="font-size: 1.5em; color: #00F0FF;">{gpu_name}</div>
            </div>
            """, unsafe_allow_html=True)
        st.markdown(f"""
            <div class="glowing-card">
                <div data-testid="stMetricLabel">System Uptime</div>
                <div data-testid="stMetricValue" style="font-size: 1.5em;">{get_system_uptime()}</div>
            </div>
            """, unsafe_allow_html=True)

    with col2:
        st.subheader("Live VRAM")
        if 'gpu_history' not in st.session_state: st.session_state.gpu_history = []
        st.session_state.gpu_history.append({"time": datetime.now(), "VRAM (GB)": used_mem, "Total (GB)": total_mem})
        st.session_state.gpu_history = st.session_state.gpu_history[-50:]

        if st.session_state.gpu_history and total_mem > 0:
            df = pd.DataFrame(st.session_state.gpu_history)
            fig = px.area(df, x="time", y="VRAM (GB)", title="VRAM Usage Over Time", range_y=[0, total_mem])
            fig.update_layout(template="plotly_dark", paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font_color="#00F0FF", xaxis=dict(showgrid=False), yaxis=dict(gridcolor='#444'))
            fig.update_traces(line=dict(color="#FF00FF"))
            st.plotly_chart(fig, use_container_width=True, config={'displayModeBar': False})
        else:
            st.info("No GPU detected. VRAM chart disabled.")

    st.markdown("---")

    # --- Logs, Analytics, and Config in Tabs ---
    log_tab, analytics_tab, config_tab = st.tabs(["[ Terminal Logs ]", "[ User Analytics ]", "[ System Configuration ]"])

    with log_tab:
        st.subheader("Live System Logs")
        st.markdown(f"""
            <div style="height: 300px; background-color: #0A0C12; border-radius: 5px; padding: 10px; overflow-y: scroll; border: 1px solid #444;">
                {''.join(f'<div class="log-entry">{log}</div>' for log in st.session_state.system_logs)}
            </div>
        """, unsafe_allow_html=True)

    with analytics_tab:
        st.subheader("User Feedback & Usage")
        trending_queries_df = get_trending_queries()
        lang_usage_df = get_language_usage()

        colA, colB = st.columns(2)
        with colA:
            st.subheader("Trending Queries (Top 5)")
            if not trending_queries_df.empty:
                fig_trends = px.bar(trending_queries_df, x='Count', y='Query', title="Trending Queries", orientation='h')
                fig_trends.update_layout(template="plotly_dark", paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font_color="#FF00FF")
                st.plotly_chart(fig_trends, use_container_width=True)
            else:
                st.info("No queries logged yet.")
        with colB:
            st.subheader("Language Usage Breakdown")
            if not lang_usage_df.empty and lang_usage_df['Count'].sum() > 0:
                fig_lang = px.bar(lang_usage_df, x='Language', y='Count', title='Language Usage', color='Language')
                fig_lang.update_layout(template="plotly_dark", paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', font_color="#00F0FF")
                st.plotly_chart(fig_lang, use_container_width=True)
            else:
                 st.info("No language usage data recorded yet.")

    with config_tab:
        st.subheader("🔐 Token Manager")
        st.session_state.reveal_keys = st.checkbox("👁️ Reveal Keys", value=st.session_state.reveal_keys)
        key_type = "text" if st.session_state.reveal_keys else "password"
        st.text_input("HF_TOKEN", value=HF_TOKEN, type=key_type, disabled=True)
        st.text_input("JWT_SECRET_KEY", value=JWT_SECRET, type=key_type, disabled=True)
        st.text_input("NGROK_TOKEN", value=NGROK_TOKEN, type=key_type, disabled=True)
        st.info("Tokens are loaded from Colab Secrets and cannot be edited here.")

        st.markdown("---")
        st.subheader("🧪 Experimental Controls")
        st.session_state.exp_mixed_precision = st.toggle("Mixed Precision (AMP)", value=st.session_state.exp_mixed_precision, help="Use torch.cuda.amp.autocast for inference. (Default: On)")
        st.session_state.exp_torch_compile = st.toggle("Torch.compile() (Experimental)", value=st.session_state.exp_torch_compile, help="Use torch.compile() for potential speedup. Requires restart. (Default: Off)")

        st.markdown("---")
        st.subheader("🧠 Model Hot-Swap & Memory")
        st.markdown("<h6>Model Hot-Swap</h6>", unsafe_allow_html=True)
        m1, m2, m3 = st.columns(3)
        if m1.button("Load Gemma-2B", use_container_width=True):
            get_model_and_tokenizer("Gemma-2B-IT")
            st.rerun()
        if m2.button("Load DeepSeek", use_container_width=True):
            get_model_and_tokenizer("DeepSeek-Coder-1.3B")
            st.rerun()
        if m3.button("Load Phi-2", use_container_width=True):
            get_model_and_tokenizer("Phi-2")
            st.rerun()

        st.markdown("<h6>Memory Controls</h6>", unsafe_allow_html=True)
        c1, c2 = st.columns(2)
        if c1.button("♻️ Clear VRAM Cache", use_container_width=True):
            with st.spinner("Clearing cache..."):
                gc.collect()
                torch.cuda.empty_cache()
                add_system_log("VRAM Cache Cleared", "🧹")
                st.toast("VRAM Cache Cleared!")
        if c2.button("🔄 Reload Current Model", use_container_width=True):
            if 'loaded_model_name' in st.session_state and st.session_state.loaded_model_name:
                add_system_log(f"Reloading model: {st.session_state.loaded_model_name}", "🔄")
                current_model_key = [k for k, v in MODELS.items() if v == st.session_state.loaded_model_name][0]
                del st.session_state.loaded_model_name
                if 'loaded_model' in st.session_state: del st.session_state.loaded_model
                if 'loaded_tokenizer' in st.session_state: del st.session_state.loaded_tokenizer
                get_model_and_tokenizer(current_model_key) # This will force a reload
                st.toast("Model reloaded!")
                st.rerun()
            else:
                st.toast("No model loaded to reload.")


def show_admin_users_page():
    if st.session_state.role != 'admin':
        st.error("You do not have permission to view this page.")
        return
    st.title("👑 Admin: User Management")

    tab_users, tab_history, tab_feedback = st.tabs(["User List", "Full Query History", "Full Feedback Log"])

    with tab_users:
        st.subheader("All Registered Users")
        users = get_all_users()
        if users:
            df_data = [{"ID": user[0], "Username": user[1], "Role": user[2]} for user in users]
            st.dataframe(pd.DataFrame(df_data), use_container_width=True, hide_index=True)
        else:
            st.info("No users found (except you).")

    with tab_history:
        st.subheader("Full Query History (Last 20)")
        st.dataframe(get_full_history(), use_container_width=True)

    with tab_feedback:
        st.subheader("Full Feedback Log (Last 10)")
        feedback_data = get_full_feedback()
        if not feedback_data.empty:
            st.dataframe(feedback_data, use_container_width=True)
        else:
            st.info("No feedback has been submitted yet.")


# --- 8. USER-FACING PAGES (Unchanged) ---

def show_profile_page():
    st.title(f"Profile: {st.session_state.username}")
    st.info(f"**Username:** `{st.session_state.username}`\n\n**Role:** `{st.session_state.role}`")
    st.subheader("Change Password")
    with st.form("change_password_form"):
        current_password = st.text_input("Current Password", type="password")
        new_password = st.text_input("New Password", type="password")
        confirm_new_password = st.text_input("Confirm New Password", type="password")
        submit_change = st.form_submit_button("Update Password")
        if submit_change:
            user_data = verify_user(st.session_state.username, current_password)
            if not user_data:
                st.error("Incorrect current password.")
            elif not new_password or new_password != confirm_new_password:
                st.error("New passwords do not match or are empty.")
            else:
                update_user_password(st.session_state.username, new_password)
                add_system_log(f"User '{st.session_state.username}' changed password.", "🔑")
                st.success("Password updated successfully!")

    st.markdown("---")
    st.subheader("Recent Activity History")
    history_df = get_user_history(st.session_state.username)
    if not history_df.empty:
        st.dataframe(history_df, use_container_width=True, hide_index=True)
    else:
        st.info("No activity logged yet.")

def preload_selected_model():
    if 'gen_model_key' in st.session_state and st.session_state.gen_model_key:
        print(f"Pre-loading model: {st.session_state.gen_model_key}")
        get_model_and_tokenizer(st.session_state.gen_model_key)

def show_generator_page():
    st.title("🤖 Code Generator")
    model_keys = list(MODELS.keys())
    selected_model = st.selectbox(
        "Select AI Model",
        model_keys,
        key='gen_model_key',
        on_change=preload_selected_model
    )

    if 'loaded_model' not in st.session_state or st.session_state.loaded_model is None:
        preload_selected_model()

    chat_container = st.container(height=400, border=True)
    with chat_container:
        for entry in st.session_state.chat_history:
            with st.chat_message(entry["role"]):
                st.markdown(entry["content"])
    prompt = st.text_area("Your prompt:", key="prompt_input", height=100)
    col1, col2, col3 = st.columns([2, 1, 1])

    with col1:
        if st.button("Generate Code 🚀", use_container_width=True, type="primary"):
            if prompt:
                st.session_state.chat_history.append({"role": "user", "content": prompt})
                with chat_container:
                    with st.chat_message("user"):
                        st.markdown(prompt)
                    with st.chat_message("assistant"):
                         message_placeholder = st.empty()

                with st.spinner(f"CodeGenie is thinking with {selected_model}..."):
                    start_time = time.time()
                    try:
                        response = generate_code(prompt, selected_model)
                    except Exception as e:
                        response = f"Error: {str(e)}"
                    end_time = time.time()
                    latency = end_time - start_time

                log_user_activity(st.session_state.username, prompt, selected_model, "Python", latency)
                message_placeholder.markdown(response)
                st.session_state.chat_history.append({"role": "assistant", "content": response})
                if not response.startswith("Error:"):
                    st.session_state.last_output = response
                st.rerun()
            else:
                st.warning("Please enter a prompt.")

    with col2:
        if st.button("📋 Copy Last Output", use_container_width=True):
            if st.session_state.last_output:
                st.components.v1.html(f"<script>navigator.clipboard.writeText({repr(st.session_state.last_output)});</script>", height=0)
                st.toast("Output copied to clipboard!")
            else:
                st.toast("No output to copy yet.")
    with col3:
        if st.button("🗑️ Clear History", use_container_width=True):
            st.session_state.chat_history = []
            st.session_state.last_output = ""
            st.session_state.explanation = ""
            st.session_state.ast_graph = None
            st.rerun()

    if 'last_activity_id' in st.session_state and st.session_state.last_activity_id is not None:
        st.markdown("---")
        with st.form(key="feedback_form_gen"):
            st.subheader("Was this response helpful?")
            rating = st.radio(
                "Rate this response:",
                options=[1, 2, 3, 4, 5],
                format_func=lambda x: "⭐" * x,
                horizontal=True
            )
            comment = st.text_area("Additional feedback (optional):")

            if st.form_submit_button("Submit Feedback"):
                try:
                    update_feedback_score(st.session_state.last_activity_id, rating, comment)
                    st.success("Thank you for your feedback!")
                    st.session_state.last_activity_id = None
                    st.rerun()
                except Exception as e:
                    st.error(f"Failed to submit feedback: {e}")

def preload_explainer_model():
    if 'exp_model_key' in st.session_state and st.session_state.exp_model_key:
        print(f"Pre-loading explainer model: {st.session_state.exp_model_key}")
        get_model_and_tokenizer(st.session_state.exp_model_key)

def show_explainer_page():
    st.title("🔬 Code Explainer & AST Visualizer")
    lang = st.selectbox("Select Language", ["Python", "JavaScript", "SQL"])
    code = st.text_area("Paste your code here:", height=250)
    model_keys = list(MODELS.keys())
    explainer_model_name = st.selectbox(
        "Select Explainer Model",
        model_keys,
        index=0,
        key='exp_model_key',
        on_change=preload_explainer_model
    )

    if 'loaded_model' not in st.session_state or st.session_state.loaded_model is None:
        preload_explainer_model()

    if st.button("Explain & Visualize", type="primary"):
        if not code:
            st.warning("Please paste some code to explain.")
            st.session_state.explanation = ""
            st.session_state.ast_graph = None
            return

        st.session_state.explanation = ""
        st.session_state.ast_graph = None

        with st.spinner(f"Generating explanation with {explainer_model_name}..."):
            explain_prompt = f"Explain the following {lang} code. Describe what it does, step by step. Provide the explanation in markdown format, using code blocks for the code.\n\n```\n{code}\n```"
            start_time = time.time()
            try:
                explanation = query_hf_model(explain_prompt, explainer_model_name)
            except Exception as e:
                explanation = f"Error: {str(e)}"
            end_time = time.time()
            latency = end_time - start_time
            log_user_activity(st.session_state.username, code, explainer_model_name, lang, latency)
            st.session_state.explanation = explanation

        if lang == "Python":
            visualizer = ASTVisualizer()
            graph = visualizer.get_graph(code)
            if graph:
                st.session_state.ast_graph = graph

        st.session_state.last_output = "Explainer Run"

    if st.session_state.explanation:
        col1, col2 = st.columns(2)
        with col1:
            st.subheader("AI Explanation")
            st.markdown(st.session_state.explanation)
        with col2:
            st.subheader("Abstract Syntax Tree (AST)")
            if st.session_state.ast_graph:
                st.graphviz_chart(st.session_state.ast_graph)
            elif lang == "Python":
                 st.error("Could not generate AST for this Python code.")
            else:
                st.info(f"AST visualization is only available for Python.")

    if 'last_activity_id' in st.session_state and st.session_state.last_activity_id is not None and st.session_state.last_output == "Explainer Run":
        st.markdown("---")
        with st.form(key="feedback_form_exp"):
            st.subheader("Was this explanation helpful?")
            rating = st.radio(
                "Rate this response:",
                options=[1, 2, 3, 4, 5],
                format_func=lambda x: "⭐" * x,
                horizontal=True
            )
            comment = st.text_area("Additional feedback (optional):")

            if st.form_submit_button("Submit Feedback"):
                try:
                    update_feedback_score(st.session_state.last_activity_id, rating, comment)
                    st.success("Thank you for your feedback!")
                    st.session_state.last_activity_id = None
                    st.rerun()
                except Exception as e:
                    st.error(f"Failed to submit feedback: {e}")


# --- 10. MAIN APP ROUTER (WITH ROLE-BASED NAVIGATION) ---

def main():
    st.set_page_config(page_title="CodeGenie", page_icon="🧞", layout="wide")
    is_first_user = init_db()
    initialize_state()

    if st.session_state.token:
        payload = verify_jwt(st.session_state.token)
        if payload:
            st.session_state.logged_in = True
            st.session_state.username = payload['sub']
            st.session_state.role = payload['role']
        else:
            for key in list(st.session_state.keys()):
                del st.session_state[key]
            st.rerun()

    if not st.session_state.logged_in:
        if st.session_state.page == "Forgot Password":
            show_forgot_password_page()
        else:
            show_login_page(is_first_user)
    else:
        with st.sidebar:
            st.title(f"CodeGenie 🧞")
            st.markdown(f"Welcome, **{st.session_state.username}**!")
            st.markdown("---")
            def set_page(page_name):
                st.session_state.page = page_name

            if st.session_state.role == 'admin':
                st.markdown("*Admin Tools*")
                st.button("Admin Dashboard", use_container_width=True, type="primary" if st.session_state.page == "Dashboard" else "secondary", on_click=set_page, args=("Dashboard",))
                st.button("Manage Users", use_container_width=True, type="primary" if st.session_state.page == "Admin" else "secondary", on_click=set_page, args=("Admin",))
                st.button("My Profile", use_container_width=True, type="primary" if st.session_state.page == "Profile" else "secondary", on_click=set_page, args=("Profile",))

            else: # If role == 'user'
                st.button("Code Generator", use_container_width=True, type="primary" if st.session_state.page == "Generator" else "secondary", on_click=set_page, args=("Generator",))
                st.button("Code Explainer", use_container_width=True, type="primary" if st.session_state.page == "Explainer" else "secondary", on_click=set_page, args=("Explainer",))
                st.button("My Profile", use_container_width=True, type="primary" if st.session_state.page == "Profile" else "secondary", on_click=set_page, args=("Profile",))

            st.markdown("---")
            if st.button("Logout 🔒", use_container_width=True):
                add_system_log(f"User '{st.session_state.username}' logged out.", "👋")
                for key in list(st.session_state.keys()):
                    del st.session_state[key]
                st.rerun()

        if st.session_state.role == 'admin':
            if st.session_state.page == "Dashboard":
                show_admin_dashboard_page()
            elif st.session_state.page == "Admin":
                show_admin_users_page()
            elif st.session_state.page == "Profile":
                show_profile_page()
            else:
                st.session_state.page = "Dashboard"
                st.rerun()

        else: # If role == 'user'
            if st.session_state.page == "Generator":
                show_generator_page()
            elif st.session_state.page == "Explainer":
                show_explainer_page()
            elif st.session_state.page == "Profile":
                show_profile_page()
            else:
                st.session_state.page = "Generator"
                st.rerun()

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
# Cell 3: Launch the App (Final Version)
import os
import time
import subprocess
import socket
from google.colab import userdata
from pyngrok import ngrok

# --- Clean up any old ngrok/Streamlit processes ----
print("🧹 Cleaning up old ngrok and Streamlit processes...")
!pkill -f ngrok 2>/dev/null || echo "No old ngrok"
!kill -9 $(lsof -t -i:8501) 2>/dev/null || echo "No old Streamlit"

# ---- Authenticate ngrok ----
ngrok.set_auth_token(os.environ['NGROK_TOKEN'])

# ---- Helper to wait for Streamlit ----
def wait_for_port(host="localhost", port=8501, timeout=90):
    start = time.time()
    while time.time() - start < timeout:
        try:
            with socket.create_connection((host, port), timeout=2):
                return True
        except OSError:
            time.sleep(1)
    return False

# ---- Start Streamlit ----
print("🚀 Starting Streamlit on port 8501...")
# Start the process and pipe logs to files
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8501", "--server.headless", "true"],
    stdout=open("streamlit.log", "w"),
    stderr=open("streamlit.err", "w")
)

# ---- Wait until Streamlit is ready ----
print("⏳ Waiting for Streamlit to start...")
if not wait_for_port():
    print("❌ Streamlit did not start. Check app logs.")
    # Print the error logs if it fails
    !echo "--- STDOUT ---"
    !cat streamlit.log
    !echo "--- STDERR ---"
    !cat streamlit.err
else:
    # ---- Open ngrok tunnel only after Streamlit is ready ----
    print("🌐 Opening ngrok tunnel...")
    tunnel = ngrok.connect(8501)
    print(f"\n✅ Your Streamlit app is live!\n👉 {tunnel.public_url}\n")

🧹 Cleaning up old ngrok and Streamlit processes...
^C
🚀 Starting Streamlit on port 8501...
⏳ Waiting for Streamlit to start...
🌐 Opening ngrok tunnel...

✅ Your Streamlit app is live!
👉 https://rosemary-tricuspidate-earlene.ngrok-free.dev

